In [2]:
from nilearn.connectome import ConnectivityMeasure
from nilearn.maskers import MultiNiftiLabelsMasker
from nilearn import plotting
from nilearn import datasets

In [3]:
schaefer = datasets.fetch_atlas_schaefer_2018(n_rois=100, yeo_networks=7, resolution_mm=1, data_dir=None, base_url=None, resume=True, verbose=1)
print(
    "Yeo atlas nifti image (3D) with 400 ROIs and 7 networks is located at: "
    f" is located at: {schaefer.maps}"
)

[get_dataset_dir] Dataset found in /Users/aj/nilearn_data/schaefer_2018
Yeo atlas nifti image (3D) with 400 ROIs and 7 networks is located at:  is located at: /Users/aj/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_1mm.nii.gz


In [4]:
schaefer_labels = schaefer.labels
print(schaefer_labels)

[b'7Networks_LH_Vis_1' b'7Networks_LH_Vis_2' b'7Networks_LH_Vis_3'
 b'7Networks_LH_Vis_4' b'7Networks_LH_Vis_5' b'7Networks_LH_Vis_6'
 b'7Networks_LH_Vis_7' b'7Networks_LH_Vis_8' b'7Networks_LH_Vis_9'
 b'7Networks_LH_SomMot_1' b'7Networks_LH_SomMot_2'
 b'7Networks_LH_SomMot_3' b'7Networks_LH_SomMot_4'
 b'7Networks_LH_SomMot_5' b'7Networks_LH_SomMot_6'
 b'7Networks_LH_DorsAttn_Post_1' b'7Networks_LH_DorsAttn_Post_2'
 b'7Networks_LH_DorsAttn_Post_3' b'7Networks_LH_DorsAttn_Post_4'
 b'7Networks_LH_DorsAttn_Post_5' b'7Networks_LH_DorsAttn_Post_6'
 b'7Networks_LH_DorsAttn_PrCv_1' b'7Networks_LH_DorsAttn_FEF_1'
 b'7Networks_LH_SalVentAttn_ParOper_1'
 b'7Networks_LH_SalVentAttn_FrOperIns_1'
 b'7Networks_LH_SalVentAttn_FrOperIns_2'
 b'7Networks_LH_SalVentAttn_PFCl_1' b'7Networks_LH_SalVentAttn_Med_1'
 b'7Networks_LH_SalVentAttn_Med_2' b'7Networks_LH_SalVentAttn_Med_3'
 b'7Networks_LH_Limbic_OFC_1' b'7Networks_LH_Limbic_TempPole_1'
 b'7Networks_LH_Limbic_TempPole_2' b'7Networks_LH_Cont_Par_1'
 

In [5]:
print(schaefer.description)

.. _schaefer_atlas:

Schaefer 2018 atlas

Access
------
See :func:`nilearn.datasets.fetch_atlas_schaefer_2018`.

Notes
-----
This atlas (:footcite:t:`schaefer_atlas`) provides a labeling of cortical voxels in the MNI152
space, see :footcite:t:`Schaefer2017`.
Each ROI is annotated with a network from the :term:`parcellation`
(7- or 17-network solution; see :footcite:t:`Yeo2011`).

Different versions of the atlas are available, varying in
- number of rois (100 to 1000),
- network annotation (7 or 17)
- spatial resolution of the atlas (1 or 2 mm)

Content
-------
    :'maps': 3D Nifti image, values are indices in the list of labels.
    :'labels': ROI labels including Yeo-network annotation.
    :'description': A short description of the atlas and some references.

References
----------

.. footbibliography::

License
-------
MIT



In [6]:
data = datasets.fetch_development_fmri(n_subjects=10)

print(
    "Functional nifti images (4D, e.g., one subject) "
    f"are located at : {data.func[0]!r}"
)
print(
    "Counfound csv files (of same subject) are located "
    f"at : {data['confounds'][0]!r}"
)

[get_dataset_dir] Dataset found in /Users/aj/nilearn_data/development_fmri
[get_dataset_dir] Dataset found in /Users/aj/nilearn_data/development_fmri/development_fmri
[get_dataset_dir] Dataset found in /Users/aj/nilearn_data/development_fmri/development_fmri
Functional nifti images (4D, e.g., one subject) are located at : '/Users/aj/nilearn_data/development_fmri/development_fmri/sub-pixar123_task-pixar_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
Counfound csv files (of same subject) are located at : '/Users/aj/nilearn_data/development_fmri/development_fmri/sub-pixar123_task-pixar_desc-reducedConfounds_regressors.tsv'


In [7]:
# ConenctivityMeasure from Nilearn uses simple 'correlation' to compute
# connectivity matrices for all subjects in a list
connectome_measure = ConnectivityMeasure(
    kind="correlation",
    standardize="zscore_sample",
)

# create masker using MultiNiftiLabelsMasker to extract functional data within
# atlas parcels from multiple subjects using parallelization to speed up the
# computation
masker = MultiNiftiLabelsMasker(
    labels_img=schaefer.maps,  # Both hemispheres
    standardize="zscore_sample",
    standardize_confounds="zscore_sample",
    memory="nilearn_cache",
    n_jobs=2,
)

# extract time series from all subjects
time_series = masker.fit_transform(data.func, confounds=data.confounds)

# calculate correlation matrices across subjects and display
correlation_matrices = connectome_measure.fit_transform(time_series)

# Mean correlation matrix across 10 subjects can be grabbed like this,
# using connectome measure object
mean_correlation_matrix = connectome_measure.mean_

# grab center coordinates for atlas labels
coordinates = plotting.find_parcellation_cut_coords(labels_img=schaefer.maps)

In [1]:
# plot connectome with 80% edge strength in the connectivity
connectome = plotting.plot_connectome(
    mean_correlation_matrix, coordinates, edge_threshold="95%", node_size=5, colorbar=True
)

NameError: name 'plotting' is not defined